In [485]:
import pandas as pd

In [486]:
import numpy as np

In [487]:
test = pd.read_csv("data/test.tsv", sep="\t", index_col="PhraseId")

# print(test.shape)
# test.head(10)

In [488]:
train = pd.read_csv("data/train.tsv", sep="\t", index_col="PhraseId")

# print(train.shape)
# train.head(10)

## Preprocessing

In [489]:
train["Phrase(Origin)"] =  train["Phrase"].copy()

# print(train.shape)
# train[["Phrase", "Phrase(Origin)"]].head()

In [514]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from tqdm import tqdm

import string

lem = WordNetLemmatizer()
lem

<WordNetLemmatizer>

In [538]:
def clean_text(phrase):
    phrase = phrase.replace("ca n't", "can not")
    phrase = phrase.replace("n't", "not")
    phrase = phrase.replace("'s", " is")
    
    ### homework
#     lem_words = [lem.lemmatize(w) for w in phrase.split(" ")]
    lem_words = [lem.lemmatize(w) for w in word_tokenize(phrase) if w not in string.punctuation]
    lem_pharse = " ".join(lem_words)
    phrase = lem_pharse
    
    return phrase
    
train["Phrase"] = train["Phrase"].apply(clean_text)
# tqdm.pandas(desc="cleansing ...")
# train["Phrase"] = train["Phrase"].progress_apply(clean_text)

print(train.shape)
train[["Phrase", "Phrase(Origin)"]].head()

(156060, 4)


,Phrase,Phrase(Origin)
PhraseId,,
1,A series of escapade demonstrating the adage t...,A series of escapades demonstrating the adage ...
2,A series of escapade demonstrating the adage t...,A series of escapades demonstrating the adage ...
3,A series,A series
4,A,A
5,series,series


In [516]:
import nltk
import string

def tokenize(text):
    text = ''.join([ch for ch in text])# if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

In [517]:
test["Phrase(Origin)"] =  test["Phrase"].copy()
test["Phrase"] = test["Phrase"].apply(clean_text)

# print(test.shape)
# test[["Phrase", "Phrase(Origin)"]].head()

## Vectorize phrase

In [518]:
from sklearn.feature_extraction.text import CountVectorizer

#vectorizer = CountVectorizer(max_features=100) # default(1,1)
# vectorizer = CountVectorizer(max_features=20000,
#                              ngram_range=(1, 3),
# #                              tokenizer=tokenize,
#                              lowercase = True,
#                              max_df = 0.5,
#                              min_df = 10,
#                              stop_words='english')
vectorizer = CountVectorizer(max_features=20000, ngram_range=(1, 3)), max_df = 0.3)

# vectorizer

In [519]:
# fit /

vectorizer.fit(train["Phrase"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [520]:
X_train = vectorizer.transform(train["Phrase"])
# X_train

In [521]:
vocabulary = vectorizer.get_feature_names()
pd.DataFrame(X_train[0:1000].toarray(), columns=vocabulary).head()

,000,10,10 minute,10 or,10 or 15,10 year,100,100 minute,100 year,101,...,zany,zeal,zealand,zero,zhang,zhang is,zinger,zippy,zombie,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [522]:
X_test = vectorizer.transform(test["Phrase"])
# X_test

In [523]:
y_train = train["Sentiment"]

# print(y_train.shape)
# y_train.head()

In [524]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier()
# model = SGDClassifier(random_state=40)
# model

In [525]:
# fit(학습 - train) / predict(예측 - test data)
model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [526]:
predictions = model.predict(X_test)

# print(predictions.shape)
# predictions

## Score

In [527]:
# Cross-validation
from sklearn.model_selection import cross_val_score

score = cross_val_score(model, X_train, y_train).mean()
print("Score = {0:.6f}".format(score))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\sit

Score = 0.576970


## Submit

In [528]:
submit = pd.read_csv("data/sampleSubmission.csv", index_col="PhraseId")

print(submit.shape)
submit.head()

(66292, 1)


,Sentiment
PhraseId,
156061,2
156062,2
156063,2
156064,2
156065,2


In [529]:
submit["Sentiment"] = predictions

print(submit.shape)
submit.head()

(66292, 1)


,Sentiment
PhraseId,
156061,3
156062,3
156063,2
156064,3
156065,3


In [530]:
submit.to_csv("data/baseline-script.csv")

In [531]:
from sklearn.model_selection import cross_val_predict

y_predict = cross_val_predict(model, X_train, y_train, cv=5)

print(y_predict.shape)
y_predict

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\sit

(156060,)


array([1, 2, 2, ..., 2, 2, 2], dtype=int64)

In [532]:
result = train.copy()
result["Sentiment(Predict)"] = y_predict

result["Distance"] = \
  np.abs(result["Sentiment"] - result["Sentiment(Predict)"])
    
result = result.sort_values(by="Distance", ascending=False)

print(result.shape)
result.head()

(156060, 6)


,SentenceId,Phrase,Sentiment,Phrase(Origin),Sentiment(Predict),Distance
PhraseId,,,,,,
73392,3751,a the filmmaker look down on their working-cla...,0,as the filmmakers look down on their working-c...,4,4
91126,4741,enjoys intentionally introducing your kid to f...,0,enjoys intentionally introducing your kids to ...,4,4
59223,2989,post-September 11 `` The Sum Of All Fears '' s...,0,"post-September 11 , `` The Sum Of All Fears ''...",4,4
124283,6674,will find their humor-seeking dollar best spen...,0,will find their humor-seeking dollars best spe...,4,4
61625,3115,There is no substitute for on-screen chemistry...,4,There is no substitute for on-screen chemistry...,0,4


In [533]:
result.to_csv("data/result.csv")

In [534]:
vocabulary = vectorizer.get_feature_names()
pd.DataFrame(vocabulary).to_csv("data/vocabulary.csv")

In [535]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_train, y_predict)
print("Score = {0:.6f}".format(score))

Score = 0.577406


## N-GRAM

In [536]:
#vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 2)) # default(1,1)

In [537]:
# train[train["Phrase"].str.contains("can not")]

## Homework